<a href="https://colab.research.google.com/github/anupkkumarfb/LSTM/blob/master/LSTM_IMDB_Anup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('dark_background')
from keras.datasets import imdb
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, GlobalAveragePooling1D, Dense, LSTM


np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a,allow_pickle=True)
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
np.load = np_load_old


print ("The first review is : {}, its length is :{}, its type is: {}".format(X_train[1],len(X_train[1]), type(X_train)))

#Padding to make sure all reviews are of equal length
max_review_length = 500

X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
#print ("The first review is : {}, shape is: {}".format(X_train[1],X_train.shape))

embedding_vector_length = 32  
#Embedding layer - given any indices returns a vector. can be thought of word2vec.  Embedding_vector_length = number of activation functions

#Sequential model --> Output of one layer goes as input to the next
model = Sequential()

#Adding an embedding layer
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))

from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))  #Each LSTM cell will take 32 outputs from previous layer (activation function) as input
#model.add(Dense(1,activation='sigmoid'))
model.add(Dense(100,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())


import time
start = time.time()
epochs = 10

model.fit(X_train, y_train, batch_size=32, epochs=epochs)
print ("Time to train the model with {} epochs: {}".format(epochs,(time.time()-start)))

start = time.time()

scores = model.evaluate(X_test, y_test)
print ("Time to evaluate the model with {}".format((time.time()-start)))
print("Accuracy: %.2f%%" % (scores[1]*100))
print("Scores: ", scores)


#For predicting output

print("New review: \'the movie was a great waste of my time\'")
d = imdb.get_word_index()
#print(d)
review = "Awesome movie. Excellent. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. Amazing. "

words = review.split()
review = []
for word in words:
  if word not in d: 
    print("Word: ", word)
    review.append(2)
  else:
    print("Word in else: ", word)
    review.append(d[word]+3) 
    print(d[word])

print(review)

review = sequence.pad_sequences([review],truncating='pre', padding='pre', maxlen=max_review_length)
prediction = model.predict(review)
print("Prediction (0 = negative, 1 = positive) = ", end="")
print("%0.4f%%" % (prediction[0][0]*100))

The first review is : [1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2637, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 2, 5, 2, 656, 245, 2350, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95], its length is :189, its type is: <class 'numpy.ndarray'>
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 

New review: 'the movie was a great waste of my time'
1646592/1641221 [==============================] - 0s 0us/step
Word:  Awesome
Word:  movie.
Word:  Excellent.
Word:  Amazing
[2, 2, 2, 2]
Prediction (0 = negative, 1 = positive) = 21.2508%
